## Algorithm
* Input set of documents
    - number of samples to generate
    - maximum token length (input + context + answer)
    - difficulty distribution 
    - Seed questions? 
    - randomize answer output formats
* Output : test set with questions,contexts,answer

1. Select document and part of document to frame question from (random)
2. Formulate a question,context pair contrained on output format. 
3. Identify difficulty type 
4. Evol question using evol-instruct like paradigm to improve difficulty 
    - ask for improved reasoning - (Evol instruct C Increased Reasoning Steps Prompt)
    - Add more contexts and frame question for multi hop
        - Adding more context
           1. Identiy entity from current context and retrive paras with same entity : formulate new question
           2. Identify similar paras using sentence similarity : formulate new question
           
           
#### Ideas to increase complextity

- Extend question by using info in the newly added contexts.


- Concretizing

    In RAG case,this should yeild an instruction that is related to real world use-case on concept decribed in the context. 
    For example, if context speaks about different types of instructions and their use-cases
    question will be like "I want to train a chatbot, how to form training dataset?"


- Improved reasoning
           
- Reasoning over multiple contexts
    - add context1, context2, context3, etc untill max tokens - x. And then ask model to formulate a question that would require reasoning over multiple contexts. 
    
- Condition the question - introduce new conditions
    
### Question filtering
- Follow question elimination trick from evol-instruct
- Remove phrases like 'according to given question,etc from'
- Simplify question and decrease length 
- Length upper cut of questions (~20)
    
#### Open-issues
1. How to ensure inter-document dependancy is null

A question framed might have possible answers from different documents

2. Questions framed by LLM are easily answerable. 

It feels like LLM first identifies a candidate sentence and frames question based on it. So by default the answer to questions can be located easily.

3. GPT 3.5 tends to add or increase length of question when asked to create difficult question

often this ends up as a unsually long question that contains two questions

4. Bad content : some pages / parts of document will contain bad contents like aknowledgements, examples, etc. Those should be deprioritized.
    - This can be partly handled by a good chunking mechanism - each part should contains atlest 2k tokens. 
    - Some overlap - split intelligently. 
    
    
### TODO
- garbage filtering
- take inspiration from algorithms used to clean pre-training data? 
- content filtering can be done with LLM but an algorithm that could prioritize chunks might be better.

## Functions

In [6]:
import os
import openai
import json
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
os.environ["OPENAI_API_KEY"] = json.load(open("/Users/shahules/openai-key.json"))["ikka"]

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
MODEL = "gpt-4"

In [10]:
def llm2(prompt, **kwargs):
    response = openai.ChatCompletion.create(
        model=kwargs.get("model", MODEL),
        messages=[{"role": "system", "content": prompt}],
        temperature=kwargs.get("temperature", 0),
        top_p=kwargs.get("top_p", 1),
        frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        presence_penalty=kwargs.get("presence_penalty", 0.0),
        max_tokens=kwargs.get("max_tokens", 500),
        n=kwargs.get("n", 1),
    )
    return response

In [11]:
from langchain.embeddings import OpenAIEmbeddings
Embedding = OpenAIEmbeddings()

In [12]:
def find_neighbour(text, text_list, min_similarity=0.8, top_k=3):
        text_vec = np.asarray(Embedding.embed_query(text)).reshape(1, -1)
        text_list_vec = np.asarray(
            Embedding.embed_documents(text_list)
        )
        norm = np.linalg.norm(text_list_vec, axis=1) * np.linalg.norm(
            text_vec, axis=1
        )
        similarity =  (
            np.dot(text_list_vec, text_vec.T).reshape(
                -1,
            )
            / norm
        )
        similarity  = similarity[similarity>min_similarity]
        return similarity.argsort().tolist()[-top_k:]

In [13]:

"""
split long documents to max length chunks
"""


def doc_filter(corpus,top_k=20):
    vectorizer = TfidfVectorizer(min_df=0.05,max_df=0.5)
    X = vectorizer.fit_transform(corpus)
    X = X.mean(axis=1).reshape(1,-1).argsort().tolist()[0]
    return X[-top_k:][::-1]

def split_text(text, max_length=1000):
        chunks = []
        corpus = text.split('\n')
        
        current_chunk = ""
        for chunk in corpus:
            
            if len(current_chunk + chunk)//4 < max_length:
                current_chunk = "\n".join([current_chunk,chunk])
            else:
                chunks.append(current_chunk)
                current_chunk = chunk
            
        return chunks
  


In [14]:
def read_doc(path):
    with open(path,'r') as file:
        return file.read()

## Playground

- random select document 
- Identify sections from each document

In [15]:
BLOCKLIST = ["Based on the provided context",'this context']

In [16]:
path = "../arxiv-llm/textdata"
text = read_doc("../arxiv-llm/textdata/2303.18223v11.A_Survey_of_Large_Language_Models")

In [17]:
corpus = split_text(text)
top_docs = doc_filter(corpus)

In [18]:
top_docs

[4, 7, 13, 83, 8, 84, 3, 0, 6, 82, 33, 69, 77, 80, 19, 63, 52, 42, 23, 18]

In [89]:
question_formulate = """
Your task is to formulate a question from given context satifying the rules given below:
    1.The question should be fully answered from the given context. 
    2.The question should be framed from a part of context that contains important information. It can also be from tables,code,etc.
    3.The answer to the question should not contain any links. 
    4.The question should be of moderate difficulty.
    5.The question must be reasonable and must be understood and responded by humans.
    6.Do no use phrases like 'provided context',etc in the question
    7.Avoid framing question using word "and" that can be decomposed into more than one question.
    8. The question should not contain more than 10 words, make of use of abbreviation wherever possible.
    
context:{context}
"""

context_formulate =  """\
Task: Candidate sentence extraction.
Given the question and context, extract sentences from given context that can be used to answer the question. 
Rules to follow while doing this task:
    1. Your task is not to answer the question using given context but to extract sentences from given context that can be used to answer given question.
    2. The sentences could be anywhere in the provided context, pay attention to the whole context.
    3. While extracting candidate sentences you're not allowed to make any changes to sentences from given context.
    4. If the answer is not present in context, you should only return the empty string.


question:{question}
context:\n{context}
candidate sentences:\n
""" 

context_formulatev1 = """
Please extract relevant sentences from the provided context that can potentially help answer the following question. If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase "Insufficient Information."
question:{question}
context:\n{context}
candidate sentences:\n
"""

context_formulatev2 = """
Please extract relevant sentences from the provided context that can potentially help answer the following question. If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phrase "Insufficient Information".  While extracting candidate sentences you're not allowed to make any changes to sentences from given context.


question:{question}
context:\n{context}
candidate sentences:\n
"""

answer_formulate = """
Asnwer the question using the information from the qiven context. 
question:{question}
context:{context}
answer:
"""

context_from_answer = """
Given question, context and answer. Locate the relevant information in the context from context that was to used to form the given answer. 
question:\n{question}
context:\n{context}
answer:\n{answer}
extracted context:"""


content_filtering = """
Identify if the following content is suitable for framing a high quality question. The content should be deemed unsuitable if the content is dominated by credits, examples, disclaimer, references,acknowledgments.etc.
content:\n{context}
"""
# answer_index_formulate = """
# Locate the relevant information in the context and provide the start and end indices of the text that can be used to answer the question. Keep in mind that the relevant information might be surrounded by other unrelated text. 
# You can identify the relevant portion using any relevant keywords, phrases, or patterns present in the context.
# \n\n
# question:\nWhen was Einstein born?
# context:\nAlbert Einstein (/ˈaɪnstaɪn/ EYEN-styne;[4] German: [ˈalbɛʁt ˈʔaɪnʃtaɪn] (listen); 14 March 1879 – 18 April 1955) was a German-born theoretical physicist,[5] widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century.
# answer: the answer to the given question can be located between character index 0 and 155 of given context.  
# question:\n{question}
# context:\n{context}
# answer:
# """

resoning_single_context = """
Create a multi-hop reasoning question based on the provided context. The question should require the reader to make multiple logical connections or inferences using the information available in the context. Ensure that the question can be answered entirely from the information present in the context. Do not frame questions that contains more than 15 words.
"""


resoning_multi_context = """
Create a multi-hop reasoning question based on the provided context. The question should require the reader to make multiple logical connections or inferences using the information available in the context1 and context2. Ensure that the question can be answered entirely from the information present in the contexts. Do not frame questions that contains more than 15 words.
"""

reasoning_multi_context = """
 You are a prompt rewriter. You will be provided with a question and two set of contexts namely context1 and context2.
 Your task to is to complicate the given question to improve the difficulty of answering. You should do it by rewriting it into a multi-hop reasoning question based on the provided context. The question should require the reader to make multiple logical connections or inferences using the information available in the context1 and context2. Ensure that the question can be answered entirely from the information present in the contexts. Do not frame questions that contains more than 15 words.
 
"""

reasoning = """
You are a prompt rewriter. You will be provided with a question and a long context.Your task to is to complicate the given question to improve the difficulty of answering. 
You should do complicate the question by rewriting question into a multi-hop reasoning question based on the provided context. The question should require the reader to make multiple logical connections or inferences using the information available in given context. 
Here are some strategies to create multi-hop questions:

   - Bridge related entities: Identify information that relates specific entities and frame question that can be answered only by analysing information of both entities.
   
   - Use Pronouns: identify (he, she, it, they) that refer to same entity or concepts in the context, and ask questions that would require the reader to figure out what pronouns refer to.

   - Refer to Specific Details: Mention specific details or facts from different parts of the context including tables, code, etc and ask how they are related.

   - Pose Hypothetical Scenarios: Present a hypothetical situation or scenario that requires combining different elements from the context to arrive at an answer.

Rules to follow when rewriting question:
1. Ensure that the rewritten question can be answered entirely from the information present in the contexts.
2. Do not frame questions that contains more than 15 words. Use abbreviation wherever possible.
3. Make sure the question is clear and unambiguous. 
4. phrases like 'based on the provided context','according to the context',etc are not allowed to appear in the question.

question: {question}
CONTEXTS:
{context}

Multi-hop Reasoning Question:

"""

reasoning_question_reformulte = """
Rewrite the following question to make it more indirect and shorter while retaining the essence of the original question. The goal is to create a question that conveys the same meaning but in a less direct manner.
The rewritten question should shorter so use abbreviation wherever possible.
Original Question:
{question}

Indirectly Rewritten Question:"""

In [29]:
context = corpus[40]

In [30]:
prompt_input = question_formulate.format(context=context)
output = llm2(prompt_input,temperature=0)

In [31]:
question = output['choices'][0]['message']['content']

In [32]:
question

'What are the two settings considered to evaluate the instruction following ability of fine-tuned models?'

In [50]:
top_neigh = find_neighbour(context,corpus[top_docs[i]+1:top_docs[i]+20])

NameError: name 'i' is not defined

In [ ]:
context2 = corpus[top_docs[i]+top_neigh[0]]
# full_context = "".join([f"\n\ncontext{i+1}\n\n{ctx}" for i,ctx in enumerate([context,context2])])
full_context = "\n\n".join([context,context2])

In [51]:
prompt_input = reasoning.format(question=question,context=context)

In [52]:
output = llm2(prompt_input,temperature=0)

In [53]:
questionv1 = output['choices'][0]['message']['content']


In [62]:
questionv1

'In the process of evaluating the instruction following ability of fine-tuned models, which two distinct settings are employed and what are the primary types of data each setting utilizes?'

In [82]:
q = 'What are the two major approaches to adapting pre-trained LLMs, and how do factors such as scaling the instructions and formatting design impact their performance'

In [90]:
prompt_input = reasoning_question_reformulte.format(question=q)

In [91]:
output = llm2(prompt_input,temperature=0)
output['choices'][0]['message']['content']


'How do instruction scaling and format design affect the performance of the two main pre-trained LLM adaptation methods?'

In [24]:
extracted_context

'After pre-training, LLMs can acquire the general abilities for solving various tasks. However, an increasing number of studies have shown that LLM’s abilities can be further adapted according to specific goals. In this section, we introduce two major approaches to adapting pre-trained LLMs, namely instruction tuning and alignment tuning. The former approach mainly aims to enhance (or unlock) the abilities of LLMs, while the latter approach aims to align the behaviors of LLMs with human values or preferences.'

In [86]:
prompt_input = answer_formulate.format(question=question,context=context)

In [87]:
output = llm2(prompt_input)

In [88]:
answer = output['choices'][0]['message']['content']


In [89]:
answer

'In Python, variable names can be written by using alphanumeric characters and underscores. Variable names cannot start with a number and cannot contain spaces or special characters.'

### Complicating instructions

In [27]:
## Approach 1
## includes info in context2 and adds an 'and' part to the original question

multi_context = """
You are a prompt rewriter. You will be provided with a question and two set of contexts namely context1 and context2. 
Your task is to complicate the given question in a way that answering it requires information derived from both context1 and context2. 
Follow the rules given below while rewriting the question.
    1. The rewritten question should not be very long. Use abbreviation wherever possible.
    2. The rewritten question must be reasonable and must be understood and responded by humans.
    3. The rewritten question must be fully answerable from information present in context1 and context2. 
    4. Read and understand both contexts and rewrite the question so that answering requires insight from both context1 and context2.
    5. phrases like 'based on the provided context','according to the context?',etc are not allowed to appear in the question.

question:\n{question}
context1:\n{context1}
context2:\n{context2}
"""

In [87]:
single_part = """
Please rephrase the following multi-part question into a single, shortened and comprehensive question that encompasses all the key elements.
question:\n{question}
rewritten question:"""



In [137]:
prompt_input = multi_context.format(question=question,context1=extracted_context,context2=all_contexts[2])

NameError: name 'multi_context' is not defined

In [34]:
output = llm2(prompt_input)

In [36]:
questionv1 = output['choices'][0]['message']['content']

In [37]:
questionv1

'What are the two major approaches to adapting pre-trained LLMs, and how do factors such as scaling the instructions and formatting design impact their performance?'

In [88]:
output = llm2(single_part.format(question=q),temperature=0)
questionv1_compact = output['choices'][0]['message']['content']
questionv1_compact

'What are the two main methods for adapting pre-trained Language Learning Models (LLMs) and how do elements like instruction scaling and formatting design affect their performance?'

### reasoning

In [173]:
top_docs

[4, 7, 13, 83, 8, 84, 3, 0, 6, 82, 33, 69, 77, 80, 19, 63, 52, 42, 23, 18]

In [193]:
i=5

In [194]:
prompt_input = reasoning.format(reasoning_type=resoning_single_context,context=corpus[top_docs[i]])

In [195]:
output = llm2(prompt_input)

In [196]:
output

<OpenAIObject chat.completion id=chatcmpl-7w8iLF1QO2bXaFdwQtLb8iuMefLmL at 0x7fc9e9e3c5e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "What are some deficiencies of existing prompting approaches and what strategies have been proposed to address them?",
        "role": "assistant"
      }
    }
  ],
  "created": 1694090877,
  "id": "chatcmpl-7w8iLF1QO2bXaFdwQtLb8iuMefLmL",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 1299,
    "total_tokens": 1317
  }
}

In [197]:
print(corpus[top_docs[i]])

in order to detect early abnormal issues during training.
Furthermore, it also calls for more flexible mechanisms of
hardware support or resource schedule, so as to better
organize and utilize the resources in a computing cluster.
Since it is very costly to pre-train a LLM from scratch, it
is important to design a suitable mechanisms for continu-
ally pre-training or fine-tuning the LLM based on publicly
available model checkpoints ( e.g., LLaMA [57] and Flan-
T5 [64]). For this purpose, a number of technical issues
have to be resolved, e.g., catastrophic forgetting and task
specialization. However, to date, there still lack open-source
model checkpoints for LLMs with complete pre-processing
and training logs ( e.g., the scripts to prepare the pre-training
data) for reproduction. We believe that it will be of great
value to report more technical details in open-source models
for the research of LLMs. Furthermore, it is also important

57
to develop more improvement tuning strategies th

## Concretizing 


In [288]:
## Concretizing 
Concretize_prompt = """
Concretize the following question by providing a specific, real-world scenario or example that illustrates the concept or situation described in the question. The goal is to make the question more tangible and relatable.

question:\n{question}
"""

In [289]:
data_list = []
for i in np.random.randint(0,len(top_docs),2):
    print(i)
    context = corpus[top_docs[i]]
    prompt_input = question_formulate.format(context=context)
    output = llm2(prompt_input)
    question = output['choices'][0]['message']['content']
    prompt_input = Concretize_prompt.format(question=question,context=context)
    output = llm2(prompt_input)
    question_c = output['choices'][0]['message']['content']
    data_list.append({
        "question":question,
        "complex":question_c
    })

19
17


In [290]:
np.random.randint(0,len(corpus),2)

array([35,  0])

In [291]:
data_list

[{'question': 'What are the commonly used model architectures for pre-training LLMs?',
  'complex': "In the field of natural language processing, one commonly used model architecture for pre-training Language Model (LLMs) is the Transformer model. For example, let's say a team of researchers is working on developing a chatbot that can understand and generate human-like responses. They decide to pre-train their LLM using a Transformer architecture. They feed the model with a large corpus of text data, such as books, articles, and internet forums, to learn the patterns and structures of language. This pre-training phase helps the model to capture the nuances of language and develop a strong language understanding capability. Once the pre-training is complete, the researchers fine-tune the model on a specific task, such as chatbot response generation, to make it more specialized and accurate in generating appropriate responses in a conversational context."},
 {'question': 'What is the dom

In [238]:
prompt_input = Concretize_prompt.format(question=question,context=context)

In [239]:
print(prompt_input)


Concretize the following question by providing a specific, real-world scenario or example that illustrates the concept or situation described in the question. The goal is to make the question more tangible and relatable.
Your task is to rewrite the question. You SHOULD complicate the given question using the following method:
Relate the original question to a real-life scenario and reframe the question. 
Follow the rules given below while rewriting the question.
    1. The rewritten question must be reasonable and must be understood and responded by humans.
    2. The rewritten question should be fully answerable from insights derived from the provided context. 
    3. The rewritten question should not ask for any external links. 
    4. The rewritten question must not be more than 15 words. Keep the overall length of the question short by using abbreviations wherever possible.

question:
What are the three typical emergent abilities of LLMs mentioned in the context?
context:
sizes sh

In [240]:
output = llm2(prompt_input)

In [241]:
question

'What are the three typical emergent abilities of LLMs mentioned in the context?'

In [242]:
question_c = output['choices'][0]['message']['content']

In [243]:
question_c

'What are three emergent abilities of LLMs mentioned in the context, such as in-context learning?'

In [515]:
llm2(answer_formulate.format(question=question_c,context=context))

<OpenAIObject chat.completion id=chatcmpl-7tFVdnu0QTYeSDqD8JTRqowNtox7C at 0x7fe9d180cf40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The specific sources of data used for pre-training Language Models (LLMs) such as GPT-3 and GPT-NeoX include webpages, conversation data, books and news, scientific data, and code.",
        "role": "assistant"
      }
    }
  ],
  "created": 1693402013,
  "id": "chatcmpl-7tFVdnu0QTYeSDqD8JTRqowNtox7C",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 44,
    "prompt_tokens": 1263,
    "total_tokens": 1307
  }
}

In [26]:
reasoning = """
Create a multi-hop reasoning question based on the provided context. The question should require the reader to make multiple logical connections or inferences using the information available in the context. Ensure that the question can be answered entirely from the information present in the context. Do not frame questions that contains more than 15 words.

Context:
{context}

Detailed Description and Rules:
1. Understand the Context: Read the provided context carefully and understand the information it contains. Identify key pieces of information that are scattered or indirectly related to each othe
2. Identify Multiple Steps: Think about how you can construct a question that requires the reader to connect pieces of information from different parts of the context. This may involve drawing connections between various facts or concepts.
3. Formulate a Multi-hop Question: Craft a question that necessitates the reader to make multiple logical connections or inferences based on the information provided in the context. Here are some strategies to create multi-hop questions:

   - Bridge related entities: Identify information that relates specific entities and frame question that can be answered only by analysing information of both entities.
   
   - Use Pronouns: identify (he, she, it, they) that refer to same entity or concepts in the context, and ask questions that would require the reader to figure out what pronouns refer to.

   - Refer to Specific Details: Mention specific details or facts from different parts of the context and ask how they are related.

   - Pose Hypothetical Scenarios: Present a hypothetical situation or scenario that requires combining different elements from the context to arrive at an answer.

   - Ask About Cause and Effect: Inquire about the causes or effects of specific events or actions mentioned in the context, which may involve reasoning through multiple steps.

4. Ensure Clarity: Make sure the question is clear and unambiguous. It should be evident to the reader that they need to connect multiple pieces of information to answer the question.
5. phrases like 'based on the provided context','according to the context?',etc are not allowed to appear in the question.
Multi-hop Reasoning Question:

"""

In [27]:
output = llm2(test_prompt.format(context=context))
question_r = output['choices'][0]['message']['content']

In [28]:
len(question_r.split())

15

In [29]:
question_r

'What are some challenges that need to be addressed when applying LLMs to real-world scenarios?'

In [30]:
print(context)

56
some recent work has studied the human-like characteristics
of LLMs, such as self-awareness, theory of mind (ToM), and
affective computing [603, 604]. In particular, an empirical
evaluation of ToM conducted on two classic false-belief
tasks speculates that LLMs may have ToM-like abilities
since the model in the GPT-3.5 series achieves comparable
performance with nine-year-old children in ToM task [603].
In addition, another line of work has investigated applying
LLMs into the software development domain, e.g., code
suggestion [605], code summarization [606], and automated
program repair [607]. To summarize, to assist humans by
LLMs in real-world tasks has become a significant area of
research. However, it also presents challenges. Ensuring the
accuracy of LLM-generated content, addressing biases, and
maintaining user privacy and data security are crucial con-
siderations when applying LLMs to real-world scenarios.
10 C ONCLUSION AND FUTURE DIRECTIONS
In this survey, we have reviewed

## Condition on origial question

In [63]:
condition_prompt = """
Rewrite the provided question to increase its complexity by introducing a conditional element.
The goal is to make the question more intricate by incorporating a scenario or condition that affects the context of the question.
Follow the rules given below while rewriting the question.
    1. The rewritten question should not be longer than 25 words. Use abbreviation wherever possible.
    2. The rewritten question must be reasonable and must be understood and responded by humans.
    3. The rewritten question must be fully answerable from information present context.
    4. phrases like 'provided context','according to the context?',etc are not allowed to appear in the question.
for example,
question: What are the general principles for designing prompts in LLMs?
Rewritten Question:how to apply prompt designing principles to improve LLMs performance in reasoning tasks

question:{question}
context:\n{context}
Rewritten Question"""

In [64]:
question


'What are the two settings considered to evaluate the instruction following ability of fine-tuned models?'

In [65]:
question

'What are the two settings considered to evaluate the instruction following ability of fine-tuned models?'

In [66]:
prompt_input = condition_prompt.format(question=question,context=context)
output = llm2(prompt_input,temperature=0)
question_cond = output['choices'][0]['message']['content']

In [67]:
question_cond

"Considering the evaluation of fine-tuned models' instruction following ability, what are the two settings employed and how do they differ in their approach?"

In [ ]:
how to apply prompt designing principles to improve it's performance in reasoning tasks

In [102]:
llm2(single_part.format(question=question_cond))

<OpenAIObject chat.completion id=chatcmpl-7wTXeogoap5L9TqI2PvCsXJbSlHf1 at 0x7f82ec37d3b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "What are the implications of using LLMs in educational settings and what potential issues can arise from their application?",
        "role": "assistant"
      }
    }
  ],
  "created": 1694170938,
  "id": "chatcmpl-7wTXeogoap5L9TqI2PvCsXJbSlHf1",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 22,
    "prompt_tokens": 71,
    "total_tokens": 93
  }
}

## Reformat question into conversation

In [78]:
conversation = """
Reformat the provided question into two or three separate questions that could be part of a conversation. Each question should focus on a specific aspect or subtopic related to the original question.
question:{question}

Reformatted Questions for Conversation:
"""

In [79]:
question_cond

"Considering the evaluation of fine-tuned models' instruction following ability, what are the two settings employed and how do they differ in their approach?"

In [80]:
prompt_input = conversation.format(question=question_cond)
output = llm2(prompt_input,temperature=0)
questions_conv = output['choices'][0]['message']['content']

In [81]:
questions_conv

"1. What are the two settings employed in the evaluation of fine-tuned models' instruction following ability?\n2. How do these two settings differ in their approach?"